## Playground

In [86]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import gym
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.models import Sequential

def create_sequential_model(len_input:int, len_output:int, n_hidden_layers:int=0, len_hidden_layers:int=10,):
    if n_hidden_layers==0:
        model = Sequential([Dense(units=len_output, input_shape=(len_input,), activation="relu", bias_initializer="RandomNormal" )])        
        return model
    #else model = Sequential([input_layer, hidden_layers, output_layer])

def spit_output_largest(model, input_params:np.array, len_input:int):
        #run neural net and convert input to command
        decisions = model(input_params.reshape(1,len_input), training=False)
        command_int = decisions.numpy().argmax()
        print(command_int)
        return command_int

def spit_output_all(model, input_params:np.array, len_input:int):
        #run neural net and convert input to command
        decisions = model(input_params.reshape(1,len_input), training=False)
        
        return decisions.numpy()

    
def run_simulation(n_episodes:int=5, episode_len_limit:int =100 ,name:str = "CartPole-v1"):
    env = gym.make(name)
    #run each episode
    for i_episode in range(n_episodes+1):
        observation = env.reset()
        len_output = int(str(env.action_space).split("(")[1].split(")")[0])
        model = create_sequential_model(len_input=len(observation), len_output=len_output)
        fitness = 0
        #run each step
        for t in range(episode_len_limit):
            env.render()
            action = spit_output_largest(model, observation,len(observation)) 
            observation, reward, done, info = env.step(action)
            fitness += reward
            if done:
                print("Episode finished after {} timesteps, fitness = {}".format(t+1, fitness))
                break
    env.close()

def run_simulation_walker(n_episodes:int=3, episode_len_limit:int =100 ,name:str = "BipedalWalker-v3"):
    env = gym.make(name)
    len_output = env.action_space.shape[0]
    #run each episode
    for i_episode in range(n_episodes+1):
        observation = env.reset()
        model = create_sequential_model(len_input=len(observation), len_output=len_output)
        fitness = 0
        #run each step
        for t in range(episode_len_limit):
            env.render()
            action = spit_output_all(model, observation,len(observation)).flatten() #random generation np.random.uniform(-1,1,size=4)
            observation, reward, done, info = env.step(action)
            fitness += reward
            if done:
                print("Episode finished after {} timesteps, fitness = {}".format(t+1, fitness))
                break
    env.close()

In [85]:
((np.array([[0.31811935, 0.44684052, 0.2744046,  1.6320876 ]])).flatten()).shape

(4,)

In [82]:
np.random.uniform(-1,1,size=4).shape

(4,)

In [87]:
run_simulation_walker()

(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
Episode finished after 75 timesteps, fitness = -105.96298995763622
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,

In [ ]:
#run_simulation(name="CartPole-v1")

In [ ]:
env.observation_space

In [ ]:
from gym import envs
print(envs.registry.all())